In [ ]:
# default_exp cli

In [ ]:
#export
import argparse, os, sys
import fire
from fire import core
from Bio import SeqIO
import pandas as pd
from pathlib import Path

In [ ]:
#export
from duomolog import blast
from duomolog import msa
from duomolog import hmmer

In [ ]:
#export
def Display(lines, out):
    text = "\n".join(lines) + "\n"
    out.write(text)
def main():
    commands = CLI()
    core.Display = Display
    fire.Fire(commands, name="duomolog")
    # print(say_hello("Jeremy")=="Hello Jeremy!")

# Duomolog command line options

First attempt.

This is what I have in the original.

In [ ]:
#hide
test = CLI()

In [ ]:
#hide
test.blast_v_hmmer(inFile="data/enolase.part001.fa",queryFile="data/enolase.part002.fa" )

makeblastdb -dbtype prot -in duomolog_out/in_nr.fasta
blastp -outfmt 6 -query duomolog_out/query_novel.fasta -db duomolog_out/in_nr.fasta -evalue 0.001 -max_target_seqs 1
mafft duomolog_out/in_nr.fasta
blast_not_hmmer is redundant, removing it
hmmer_not_blast is redundant, removing it
blast_intersect_hmmer is empty, removing it


In [ ]:
test.blastOut

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
0,119355,17367183,68.623,443,124,8,3,442,1,431,0.0,599
1,17367189,1706653,94.009,434,26,0,1,434,1,434,0.0,853
2,34880725,1706653,80.742,431,70,3,1,418,1,431,0.0,713
3,11999257,1706653,85.399,363,53,0,1,363,26,388,0.0,652
4,20141354,1706653,84.028,432,69,0,1,432,1,432,0.0,764
5,50420053,576098,72.769,437,117,1,3,439,2,436,0.0,649
6,13431506,21263574,86.175,434,57,2,1,433,1,432,0.0,736
7,49483034,16151617,81.422,436,77,2,1,434,1,434,0.0,714
8,6624237,1706653,86.294,394,54,0,1,394,40,433,0.0,714
9,47207875,1706653,72.304,473,92,2,1,473,1,434,0.0,701


In [ ]:
#export
class CLI(object):
    def __init__(self):
        self.blastOut = None
        self.hmmerOut = None
    def blast_v_hmmer(self, 
		inFile, 
		queryFile,
		summaryOut="duomolog_out/summary_out.txt",
		blastFile=None,
        alnFile=None,
		hmmFile=None,
        intersectOnly=False):
        os.makedirs(os.path.dirname(summaryOut), exist_ok=True)
        Path(summaryOut).touch() # should at worst guarantee output is generated
        outDir = '/'.join(summaryOut.split("/")[:-1]) +"/"
        in_nrPepDB = nrPepDB(inFile) 
        in_nrPepDB.to_csv(inFile+".nr.csv")
        in_nrFile = outDir + "in_nr.fasta"
        pepDB2Fa(in_nrPepDB,in_nrFile)

        query_nrPepDB = nrPepDB(queryFile) 
        query_nrPepDB.to_csv(queryFile+".nr.csv")
        query_novelDB, query_shared_with_inputDB = intersectInQuery(in_nrPepDB,query_nrPepDB)
        query_novel_file = outDir + "query_novel.fasta"
        query_shared_with_input_File = outDir + "query_shared_with_input.fasta"
        
        if not query_shared_with_inputDB.empty:
            pepDB2Fa(query_shared_with_inputDB,query_shared_with_input_File)
        pepDB2Fa(query_novelDB,query_novel_file)
        
        if blastFile == None:
            self.blastOut = blast.run_blast(in_nrFile, query_novel_file)
        else:
            inHeaders = in_nrPepDB["1stHeader"]
            self.blastOut = blast.load_blast(blastFile,inHeaders)
        
        if hmmFile == None:
            alnFile, alnOut = msa.run_mafft(in_nrFile)
        self.hmmerOut = hmmer.run_hmmsearch(query_novel_file, alnFile, hmmFile)

        blast_headers = set(self.blastOut["qseqid"])
        hmmer_headers = set([hit.name.decode() for hit in self.hmmerOut])
        blast_hmmer_duo = Duo("blast",blast_headers,"hmmer",hmmer_headers)
        writeOut(outDir,summaryOut,blast_hmmer_duo,query_novelDB,intersectOnly)


In [ ]:
#export
class Duo:
	def __init__(self, leftName, left, rightName, right):
		self.left_union_rightName = leftName + "_union_" + rightName
		self.left_not_rightName = leftName + "_not_" + rightName
		self.left_intersect_rightName = leftName + "_intersect_" + rightName
		self.right_not_leftName = rightName + "_not_" + leftName

		self.subsets = {
			leftName: left,
			rightName: right,
			self.left_union_rightName: left | right,
			self.left_not_rightName: left - right,
			self.left_intersect_rightName: left & right,
			self.right_not_leftName: right - left
		}

	def dropEmpty(self):
		emptySubsets = []
		for subset in self.subsets:
			if self.subsets[subset] == set():
				print(subset, "is empty, removing it")
				emptySubsets.append(subset)
		for emptySubset in emptySubsets: 
			del self.subsets[emptySubset]

	def dropRedundant(self):
		nrSubSets = []
		redundantSubsets = []
		for subset in self.subsets:
			if self.subsets[subset] not in nrSubSets:
				nrSubSets.append(self.subsets[subset])
			else:
				print(subset, "is redundant, removing it")
				redundantSubsets.append(subset)
		for redundantSubset in redundantSubsets:
			del self.subsets[redundantSubset]
	def intersectOnly(self):
		self.subsets = {self.left_intersect_rightName:self.subsets[self.left_intersect_rightName]}


In [ ]:
#export
def writeOut(outDir,summaryOut,duo,queryDB,intersectOnly):
    if intersectOnly:
        duo.intersectOnly()
    else:
        duo.dropRedundant()
    duo.dropEmpty()
    if bool(duo):
        with open(summaryOut, "w") as out:
            for subset in duo.subsets:
                subsetDB = subPepDB(queryDB, duo.subsets[subset])
                pepDB2Fa(subsetDB, outDir + f"{subset}.fa")
                for header in duo.subsets[subset]:
                    out.write(f"{header}\t{subset}")


In [ ]:
#export
def nrPepDB(faFile):
    seqDict = {}
    outDict = {"1stHeader":[],  "allHeaders":[],"record":[],"seq":[]}
    with open(faFile) as fa:
        for record in SeqIO.parse(fa, "fasta"):
            if record.seq not in seqDict:
                seqDict[record.seq] = [record]
            else:
                print(f"WARNING: {record.id} contains a redundant sequence")
                seqDict[record.seq].append(record)
    
    for seq in seqDict:
        h1 = seqDict[seq][0].id
        # allHs = ';'.join(seqDict[seq])
        allHs = ';'.join([r.id for r in seqDict[seq]])
        outDict["1stHeader"].append(h1)
        outDict["allHeaders"].append(allHs)
        outDict["record"].append(seqDict[seq][0])
        outDict["seq"].append(str(seq))

    
    outDB = pd.DataFrame.from_dict(outDict)
    outDB["1stHeader"].astype(str)
    return outDB





In [ ]:
#hide
test_nrDB = nrPepDB("data/redundantSeqs.fa")

In [ ]:
#export
def pepDB2Fa(pepDB, faFile,format="fasta"):
    SeqIO.write(pepDB["record"], faFile,format)


In [ ]:
#export
def subPepDB(pepDB, headers):
    return pepDB[pepDB["1stHeader"].isin(headers)]


In [ ]:
#export
def intersectInQuery(inDB,queryDB):
    inSeqs = set(inDB["seq"])
    querySeqs = set(queryDB["seq"])
    # print(inSeqs)
    # print(querySeqs)
    sharedSeqs = inSeqs & querySeqs
    sharedDB = queryDB[queryDB["seq"].isin(sharedSeqs)]
    outDB = queryDB[~queryDB["seq"].isin(sharedSeqs)]
    # if sharedSeqs == set():
    #     # titanic[titanic["Age"] > 35]
    #     outDB = queryDB
    #     sharedDB = None
    # else:
    #     # print(sharedSeqs)
    #     sharedDB = queryDB[queryDB["seq"].isin(sharedSeqs)]
    #     outDB = queryDB[~queryDB["seq"].isin(sharedSeqs)]

    return outDB, sharedDB 


In [ ]:
#hide
p1_nrDB = nrPepDB("data/enolase.part001.fa")
p2_nrDB = nrPepDB("data/enolase.part002.fa")

p2_novelDB, p2_shared_with_p1 = intersectInQuery(p1_nrDB,p2_nrDB)

not p2_shared_with_p1.empty

False

In [ ]:
#hide
p2_shared_with_p1

In [ ]:
#hide
p2_novelDB

,1stHeader,allHeaders,record,seq
0,119355,119355,"(M, A, V, T, I, T, W, V, K, A, R, Q, I, F, D, ...",MAVTITWVKARQIFDSRGNPTVEVDVGLSDGSYARGAVPSGASTGI...
1,17367189,17367189,"(M, S, I, L, K, L, H, A, R, E, I, F, D, S, R, ...",MSILKLHAREIFDSRGNPTVEVDLYTNKGLFRAAVPSGASTGIYEA...
2,34880725,34880725,"(M, S, I, L, K, I, H, A, R, E, I, F, D, S, R, ...",MSILKIHAREIFDSRGNPTIEVDLFTAKDVFRAAVPSGVSTGIYEA...
3,11999257,11999257,"(T, D, K, G, L, F, R, A, A, V, P, S, G, A, S, ...",TDKGLFRAAVPSGASTGIYEALELRDGDKSRYLGKGVTQAVDHINK...
4,20141354,20141354,"(M, A, M, Q, K, I, F, A, R, E, I, L, D, S, R, ...",MAMQKIFAREILDSRGNPTVEVDLHTAKGRFRAAVPSGASTGIYEA...
5,50420053,50420053,"(M, T, I, S, K, V, H, S, R, Y, V, Y, D, S, R, ...",MTISKVHSRYVYDSRGNPTVEVDVTTENGLFRAIVPSGASTGINEA...
6,13431506,13431506,"(M, A, K, I, V, K, V, I, G, R, E, I, I, D, S, ...",MAKIVKVIGREIIDSRGNPTVEAEVHLEGGFVGLAAAPSGASTGSR...
7,49483034,49483034,"(M, P, I, I, T, D, V, Y, A, R, E, V, L, D, S, ...",MPIITDVYAREVLDSRGNPTVEVEVLTESGAFGRALVPSGASTGEH...
8,6624237,6624237,"(S, T, G, I, Y, E, A, L, E, L, R, D, G, D, K, ...",STGIYEALELRDGDKSRYLGKGVLKAVNHINKEIAPALIAKNFSVV...
9,47207875,47207875,"(M, S, I, L, N, I, V, A, R, E, I, L, D, S, R, ...",MSILNIVAREILDSRGNPTVEVDLHTSKGLFRAAVPSGASTGIYEA...


In [ ]:
#hide
test_inDB = pd.DataFrame.from_dict({"seq":[1,2,3],"h":[1,2,3]})
test_queryDB = pd.DataFrame.from_dict({"seq":[2,4,6],"h":[1,2,3]})
intersectInQuery(test_inDB,test_queryDB)



{1, 2, 3}
{2, 4, 6}
{2}


(   seq  h
 1    4  2
 2    6  3,
    seq  h
 0    2  1)

In [ ]:
#hide
from nbdev.showdoc import *